In [128]:

import sys
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname

from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS

In [119]:
# set up environment
conf = SparkConf() \
.setAppName("MovieLensALS") \
.set("spark.executor.memory", "2g")
sc = SparkContext(conf=conf)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=MovieLensALS, master=local[*]) created by __init__ at <ipython-input-117-72d07dc10b2d>:5 

In [159]:
def parseRating(line):
    """
    Parses a rating record in MovieLens format userId::movieId::rating::timestamp .
    """
    fields = line.strip().split("::")
    return int(fields[3][-1]) , (int(fields[0]), int(fields[1]), float(fields[2]))

In [148]:
def parseMovie(line):
    """
    Parses a movie record in MovieLens format movieId::movieTitle .
    """
    fields = line.strip().split("::")
    return int(fields[0]), fields[1]

In [149]:
def computeRmse(model, data, n):
    """
    Compute RMSE (Root Mean Squared Error).
    """
    predictions = model.predictAll(data.map(lambda x: (x[0], x[1])))
    predictionsAndRatings = predictions.map(lambda x: ((x[0], x[1]), x[2])) \
      .join(data.map(lambda x: ((x[0], x[1]), x[2]))) \
      .values()
    return sqrt(predictionsAndRatings.map(lambda x: (x[0] - x[1]) ** 2).reduce(add) / float(n))

In [162]:
ratings = sc.textFile( "ratings.dat")
ratings =ratings.map(parseRating)

In [164]:
movies = dict(sc.textFile( "movies.dat").map(parseMovie).collect())

In [167]:
numRatings = ratings.count()
numUsers = ratings.values().map(lambda r: r[0]).distinct().count()
numMovies = ratings.values().map(lambda r: r[1]).distinct().count()
print ("Got %d ratings from %d users on %d movies." % (numRatings, numUsers, numMovies))

Got 1000209 ratings from 6040 users on 3706 movies.


In [169]:
f = open("myrating", 'r')
myratings = filter(lambda r: r[2] > 0, [parseRating(line)[1] for line in f])
f.close()

In [173]:
myRatingsRDD = sc.parallelize(myratings, 1)

In [179]:
numPartitions = 4
training = ratings.filter(lambda x: x[0] < 6) \
.values() \
.union(myRatingsRDD) \
.repartition(numPartitions) \
.cache()

validation = ratings.filter(lambda x: x[0] >= 6 and x[0] < 8) \
.values() \
.repartition(numPartitions) \
.cache()

test = ratings.filter(lambda x: x[0] >= 8).values().cache()


In [180]:
numTraining = training.count()
numValidation = validation.count()
numTest = test.count()
print ("Training: %d, validation: %d, test: %d" % (numTraining, numValidation, numTest))


Training: 602252, validation: 198919, test: 199049


In [183]:
ranks = [4,8, 12]
lambdas = [0.1,1.0, 10.0]
numIters = [10, 20,30]
bestModel = None
bestValidationRmse = float("inf")
bestRank = 0
bestLambda = -1.0
bestNumIter = -1

for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
    model = ALS.train(training, rank, numIter, lmbda)
    validationRmse = computeRmse(model, validation, numValidation)
    print ("RMSE (validation) = %f for the model trained with " % validationRmse + \
              "rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter))
    if (validationRmse < bestValidationRmse):
        bestModel = model
        bestValidationRmse = validationRmse
        bestRank = rank
        bestLambda = lmbda
        bestNumIter = numIter


RMSE (validation) = 0.889889 for the model trained with rank = 4, lambda = 0.1, and numIter = 10.
RMSE (validation) = 0.883710 for the model trained with rank = 4, lambda = 0.1, and numIter = 20.
RMSE (validation) = 0.882973 for the model trained with rank = 4, lambda = 0.1, and numIter = 30.
RMSE (validation) = 1.361320 for the model trained with rank = 4, lambda = 1.0, and numIter = 10.
RMSE (validation) = 1.361321 for the model trained with rank = 4, lambda = 1.0, and numIter = 20.
RMSE (validation) = 1.361321 for the model trained with rank = 4, lambda = 1.0, and numIter = 30.
RMSE (validation) = 3.755870 for the model trained with rank = 4, lambda = 10.0, and numIter = 10.
RMSE (validation) = 3.755870 for the model trained with rank = 4, lambda = 10.0, and numIter = 20.
RMSE (validation) = 3.755870 for the model trained with rank = 4, lambda = 10.0, and numIter = 30.
RMSE (validation) = 0.878900 for the model trained with rank = 8, lambda = 0.1, and numIter = 10.
RMSE (validation)

In [184]:
testRmse = computeRmse(bestModel, test, numTest)
 # evaluate the best model on the test set
print ("The best model was trained with rank = %d and lambda = %.1f, " % (bestRank, bestLambda) \
      + "and numIter = %d, and its RMSE on the test set is %f." % (bestNumIter, testRmse))


The best model was trained with rank = 12 and lambda = 0.1, and numIter = 30, and its RMSE on the test set is 0.867070.
